In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
import os
import pandas_profiling as pp
sb.set() # set the default Seaborn style for graphics
sb.set_style("darkgrid")
sb.set_context("poster", font_scale = .5, rc={"grid.linewidth": 0.6})

# For Support Vector Machine
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import StandardScaler

# Data Preparation

Features:

- Age | Objective Feature | age | int (days) 
- Height | Objective Feature | height | int (cm) | 
- Weight | Objective Feature | weight | float (kg) | 
- Gender | Objective Feature | gender | categorical code | 
- Systolic blood pressure | Examination Feature | ap_hi | int | 
- Diastolic blood pressure | Examination Feature | ap_lo | int | 
- Cholesterol | Examination Feature | cholesterol | 1: normal, 2: above normal, 3: well above normal | 
- Glucose | Examination Feature | gluc | 1: normal, 2: above normal, 3: well above normal | 
- Smoking | Subjective Feature | smoke | binary | 
- Alcohol intake | Subjective Feature | alco | binary | 
- Physical activity | Subjective Feature | active | binary | 
- Presence or absence of cardiovascular disease | Target Variable | cardio | binary | 

In [2]:
cardio = pd.read_csv('cardio_train.csv', sep=';')
cardio.head(10)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
5,8,21914,1,151,67.0,120,80,2,2,0,0,0,0
6,9,22113,1,157,93.0,130,80,3,1,0,0,1,0
7,12,22584,2,178,95.0,130,90,3,3,0,0,1,1
8,13,17668,1,158,71.0,110,70,1,1,0,0,1,0
9,14,19834,1,164,68.0,110,60,1,1,0,0,0,0


In [3]:
print("Data type : ", type(cardio))
print("Data dims : ", cardio.shape)

Data type :  <class 'pandas.core.frame.DataFrame'>
Data dims :  (70000, 13)


In [4]:
cardio_data = cardio.copy()

In [5]:
cardio_data = cardio_data.drop_duplicates()
# Drop id column as it doesnt help 
cardio_data = cardio_data.drop(columns='id')

# Convert age into years
cardio_data['age'] = cardio_data['age']//365

# Combine age & weight into BMI
cardio_data['bmi'] = cardio_data["weight"] / (cardio_data["height"]/100)**2
cardio_data = cardio_data.drop(columns=['weight', 'height'])

In [6]:
# Convert categorical variables into “category” data type
cat_attribs = ['gender', 'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'cardio']

# Create a list that contains numerical attributes
num_attribs = ['age', 'bmi', 'ap_hi', 'ap_lo']

# Covert the data type of categorical attributes into "category"
for cat_attrib in cat_attribs:                                # iterate over each categorical attribute
  cardio_data[cat_attrib] = cardio_data[cat_attrib].astype('category')  # convert data type

cardio_data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
age,70000.0,52.840671,6.766774,29.000000,48.000000,53.000000,58.000000,64.000000
ap_hi,70000.0,128.817286,154.011419,-150.000000,120.000000,120.000000,140.000000,16020.000000
ap_lo,70000.0,96.630414,188.472530,-70.000000,80.000000,80.000000,90.000000,11000.000000
bmi,70000.0,27.556513,6.091511,3.471784,23.875115,26.374068,30.222222,298.666667


In [7]:
#Convert gender to binary From Female:1 Male:2 -> Female:0 Male:1
dummygen = pd.get_dummies(cardio_data["gender"])
#Take first column (Female:0) 
dummygen = dummygen.drop([2], axis=1)
cardio_data = pd.concat((dummygen, cardio_data), axis=1)
cardio_data = cardio_data.drop(["gender"], axis=1)
cardio_data = cardio_data.rename(columns={1:"gender"})
cardio_data.sample(5)

,gender,age,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,bmi
65950,0,47,110,60,1,1,0,1,1,0,20.796730
55867,1,47,140,80,1,1,0,0,1,0,34.889854
68675,1,55,110,70,1,1,0,0,1,0,26.171875
9433,0,63,130,80,3,1,0,0,1,1,22.058051
27175,0,56,150,80,1,1,1,0,1,1,24.382373


In [8]:
#filter out bad bp data
filter_max = ((cardio_data["ap_hi"]>220) | (cardio_data["ap_lo"]>120))
filter_min = ((cardio_data["ap_hi"]<90) | (cardio_data["ap_lo"]<60))
cardio_data = cardio_data[~filter_max]
cardio_data = cardio_data[~filter_min]
len(cardio_data)

C:\Users\User\AppData\Local\Temp\ipykernel_17508\778566219.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cardio_data = cardio_data[~filter_min]


68452

In [9]:
# Extract feature bmi from the dataset
bmi = cardio_data['bmi']

# Drop outliers
cardio_data = cardio_data.drop(cardio_data.loc[(bmi < 10) | (bmi > 80)].index)

In [10]:
# Create dummy variables for non-binary categorical data
dummies = pd.get_dummies(cardio_data[['cholesterol','gluc']])
dummies.head(10)

,cholesterol_1,cholesterol_2,cholesterol_3,gluc_1,gluc_2,gluc_3
0,1,0,0,1,0,0
1,0,0,1,1,0,0
2,0,0,1,1,0,0
3,1,0,0,1,0,0
4,1,0,0,1,0,0
5,0,1,0,0,1,0
6,0,0,1,1,0,0
7,0,0,1,0,0,1
8,1,0,0,1,0,0
9,1,0,0,1,0,0


In [11]:
cardio_data = pd.concat([cardio_data,dummies],axis=1)
cardio_data.drop(['cholesterol','gluc'],axis=1,inplace=True)
cardio_data.rename(columns = {'cholesterol_1':'cholesterol_n','cholesterol_2':'cholesterol_an','cholesterol_3':'cholesterol_wan'}, inplace = True)
cardio_data.rename(columns = {'gluc_1':'gluc_n','gluc_2':'gluc_an','gluc_3':'gluc_wan'}, inplace = True)


In [12]:
cardio_data.sample(10)

,gender,age,ap_hi,ap_lo,smoke,alco,active,cardio,bmi,cholesterol_n,cholesterol_an,cholesterol_wan,gluc_n,gluc_an,gluc_wan
10292,1,64,120,80,0,0,0,1,23.529412,1,0,0,1,0,0
63006,1,58,150,90,0,0,1,1,26.838648,0,0,1,0,0,1
10396,0,46,120,80,0,0,1,0,25.344353,1,0,0,1,0,0
38330,1,62,160,80,0,0,0,1,27.772267,1,0,0,1,0,0
58169,1,52,130,80,0,0,1,0,28.959000,1,0,0,0,0,1
49019,1,51,110,70,0,0,1,0,22.862369,1,0,0,1,0,0
24684,0,63,120,80,0,0,1,1,27.016860,1,0,0,1,0,0
34767,1,48,140,100,0,0,1,1,32.769395,1,0,0,1,0,0
7001,1,40,120,80,0,0,1,0,26.672763,1,0,0,1,0,0
3557,1,63,150,80,0,0,0,1,26.827421,0,1,0,0,1,0


# Conduct feature scaling

In [13]:
to_be_scaled_feat = ['age', 'ap_hi', 'ap_lo','bmi']
scaler=StandardScaler()
scaler.fit(cardio_data[to_be_scaled_feat])
cardio_data[to_be_scaled_feat] = scaler.transform(cardio_data[to_be_scaled_feat])

In [14]:
cardio_data.sample(10)

,gender,age,ap_hi,ap_lo,smoke,alco,active,cardio,bmi,cholesterol_n,cholesterol_an,cholesterol_wan,gluc_n,gluc_an,gluc_wan
33570,0,-1.305063,-0.405622,-0.144666,0,0,1,0,0.571741,1,0,0,1,0,0
6109,1,-0.122950,1.414358,2.019069,0,0,1,1,0.718390,1,0,0,1,0,0
54662,1,0.172578,0.201038,0.937202,0,1,1,0,0.480326,1,0,0,1,0,0
62731,0,-0.566243,-0.405622,-0.144666,1,0,0,1,-0.527434,1,0,0,1,0,0
12034,1,0.468106,-0.405622,-0.144666,0,0,1,1,-0.891794,1,0,0,1,0,0
30833,1,0.615870,-0.405622,-0.144666,0,0,1,1,0.361736,1,0,0,1,0,0
44236,1,-1.009535,-0.405622,-0.144666,0,0,0,0,0.572727,1,0,0,1,0,0
10179,1,-1.452827,-1.618942,-0.144666,0,0,1,0,-1.857393,1,0,0,1,0,0
29261,0,0.172578,2.021018,0.937202,1,0,1,1,-0.943755,1,0,0,1,0,0
58735,0,-1.452827,0.201038,0.937202,1,0,1,0,-0.540426,1,0,0,0,0,1


## Split train_set test_set

In [15]:
target_name = 'cardio'
data_target = cardio_data[target_name] #Extract cardio column as target variable
data = cardio_data.drop([target_name], axis=1) #Drop cardio column from rest of data

Split 30% test, remaining 70% train

In [16]:
Xtrain, Ytrain, Xtest, Ytest = train_test_split(data, data_target, test_size=0.3, random_state=0)

In [17]:
Xtrain.head()

,gender,age,ap_hi,ap_lo,smoke,alco,active,bmi,cholesterol_n,cholesterol_an,cholesterol_wan,gluc_n,gluc_an,gluc_wan
30434,1,-0.418479,-0.405622,-0.144666,0,0,1,-0.408886,1,0,0,1,0,0
16936,0,1.354691,-0.405622,-0.144666,1,0,1,-0.198398,1,0,0,1,0,0
35045,1,-0.861771,0.807698,0.937202,0,0,1,1.250070,0,1,0,1,0,0
36807,1,0.172578,-0.405622,-0.144666,0,0,0,-0.682481,1,0,0,1,0,0
20368,0,0.024814,0.201038,-0.144666,1,1,0,1.096583,1,0,0,0,1,0


In [18]:
Xtest.head()

30434    0
16936    1
35045    1
36807    0
20368    0
Name: cardio, dtype: category
Categories (2, int64): [0, 1]

In [19]:
Ytrain.head()

,gender,age,ap_hi,ap_lo,smoke,alco,active,bmi,cholesterol_n,cholesterol_an,cholesterol_wan,gluc_n,gluc_an,gluc_wan
7239,1,0.024814,0.807698,0.937202,0,0,1,0.718390,1,0,0,1,0,0
30316,1,1.059163,0.807698,2.019069,0,0,0,2.363471,1,0,0,1,0,0
15137,0,-1.009535,-0.405622,-1.226534,0,0,1,-0.573797,0,0,1,0,0,1
58119,0,-2.043884,0.807698,-0.144666,0,0,0,0.205194,1,0,0,1,0,0
8889,1,0.911399,0.201038,-0.144666,0,0,1,1.363770,0,0,1,1,0,0


In [20]:
Ytest.head()

7239     1
30316    1
15137    0
58119    1
8889     1
Name: cardio, dtype: category
Categories (2, int64): [0, 1]

# SVC 

In [21]:
svc = SVC(kernel='linear',C=100, cache_size=1999)
svc.fit(Xtrain, Xtest)

pred = svc.predict(Ytrain)

print('Model accuracy score with kernel=linear c=100 hyperparameters: {0:0.4f}'.
      format(accuracy_score(Ytest, pred)))

Model accuracy score with kernel=linear c=100 hyperparameters: 0.7289


In [22]:
svc = SVC(kernel='linear', C=0.1, cache_size=1999)
svc.fit(Xtrain, Xtest)

pred = svc.predict(Ytrain)

print('Model accuracy score with kernel=linear c=0.1 hyperparameters: {0:0.4f}'.
      format(accuracy_score(Ytest, pred)))

Model accuracy score with kernel=linear c=0.1 hyperparameters: 0.7290


In [23]:
svc = SVC(kernel='rbf',C=100, cache_size=1999)
svc.fit(Xtrain, Xtest)

pred = svc.predict(Ytrain)

print('Model accuracy score with kernel=rbf c=100 hyperparameters: {0:0.4f}'.
      format(accuracy_score(Ytest, pred)))

Model accuracy score with kernel=rbf c=100 hyperparameters: 0.7301


In [24]:
svc = SVC(kernel='rbf',C=0.1, cache_size=1999)
svc.fit(Xtrain, Xtest)

pred = svc.predict(Ytrain)

print('Model accuracy score with kernel=rbf c=0.1 hyperparameters: {0:0.4f}'.
      format(accuracy_score(Ytest, pred)))

Model accuracy score with kernel=rbf c=0.1 hyperparameters: 0.7313


In [25]:
svc = SVC(kernel='poly',C=100, cache_size=1999)
svc.fit(Xtrain, Xtest)

pred = svc.predict(Ytrain)

print('Model accuracy score with kernel=polynomial c=100 hyperparameters: {0:0.4f}'.
      format(accuracy_score(Ytest, pred)))

Model accuracy score with kernel=polynomial c=100 hyperparameters: 0.7316


In [26]:
svc = SVC(kernel='poly',C=0.1, cache_size=1999)
svc.fit(Xtrain, Xtest)

pred = svc.predict(Ytrain)

print('Model accuracy score with kernel=polynomial c=0.1 hyperparameters: {0:0.4f}'.
      format(accuracy_score(Ytest, pred)))

Model accuracy score with kernel=polynomial c=0.1 hyperparameters: 0.7320


In [27]:
svc = SVC(kernel='sigmoid',C=100, cache_size=1999)
svc.fit(Xtrain, Xtest)

pred = svc.predict(Ytrain)

print('Model accuracy score with kernel=sigmoid c=100 hyperparameters: {0:0.4f}'.
      format(accuracy_score(Ytest, pred)))

Model accuracy score with kernel=sigmoid c=100 hyperparameters: 0.6330


In [28]:
svc = SVC(kernel='sigmoid',C=0.1, cache_size=1999)
svc.fit(Xtrain, Xtest)

pred = svc.predict(Ytrain)

print('Model accuracy score with kernel=sigmoid c=0.1 hyperparameters: {0:0.4f}'.
      format(accuracy_score(Ytest, pred)))

Model accuracy score with kernel=sigmoid c=0.1 hyperparameters: 0.6353


## Improve accuracy by hyperparameter tuning
### Optimal Hyperparameter via gridsearch

In [29]:
# instantiate classifier with default hyperparameters with kernel=rbf, C=1.0 and gamma=auto
svc = SVC()
# declare parameters for hyperparameter tuning
parameters = [ {'C':[1, 10, 50], 'kernel':['rbf'], 'cache_size'=1999},
               {'C':[1, 10, 50], 'kernel':['poly'], 'degree': [2,4,8,16,32], 'cache_size'=1999} 
             ]

grid_search = GridSearchCV(estimator = svc,  
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           verbose=1,
                           n_jobs=-1)

grid_search.fit(Xtrain, Xtest)

SyntaxError: invalid syntax (1382536720.py, line 4)

In [ ]:
# examine the best model


# best score achieved during the GridSearchCV
print('GridSearch CV best score : {:.4f}\n\n'.format(grid_search.best_score_))


# print parameters that give the best results
print('Parameters that give the best results :','\n\n', (grid_search.best_params_))


# print estimator that was chosen by the GridSearch
print('\n\nEstimator that was chosen by the search :','\n\n', (grid_search.best_estimator_))

In [ ]:
# calculate GridSearch CV score on test set

print('GridSearch CV score on test set: {0:0.4f}'.format(grid_search.score(Ytrain, Ytest)))